# 🎮 Kapitola 8: AI ve hrách - Minimax algoritmus

<div style="background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h2 style="color: white; margin: 0;">Blok 1 | AI pro začátečníky</h2>
    <p style="color: white; margin: 10px 0;">🎯 Praktická kapitola - Neporazitelná AI pro piškvorky</p>
</div>

## 🎯 Co se naučíte

V této kapitole se zaměříme na:

- **Tradiční herní AI** - jak počítače hrály šachy a dámu
- **Minimax algoritmus** - AI, která předpovídá všechny možné tahy
- **Alpha-Beta prořezávání** - optimalizace Minimaxu
- **Moderní herní AI** - zpětnovazební učení (Reinforcement Learning)
- **Praktický projekt** - naprogramujete neporazitelnou AI pro piškvorky!

## ⚠️ Předpoklady
Tato kapitola navazuje na: Kapitoly 1-7 (základy AI)

---

## 🔧 Instalace knihoven

Tento notebook je navržen pro **Google Colab** a využívá pouze standardní Python knihovny.

In [ ]:
# Základní importy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from IPython.display import clear_output
import time
import copy
from datetime import datetime

print("="*60)
print("🎮 KAPITOLA 8: AI ve hrách - Minimax algoritmus")
print("="*60)
print(f"📅 Datum: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("✅ Všechny knihovny načteny!")
print("="*60)

---

## 📖 Část 1: Historie AI ve hrách

### Od Pac-Mana k AlphaGo

| Rok | Milník | Hra | Typ AI |
|-----|--------|-----|--------|
| 1980 | Pac-Man | Arcade | Jednoduchá pravidla |
| 1997 | Deep Blue vs. Kasparov | Šachy | Minimax + evaluace |
| 2016 | AlphaGo vs. Lee Sedol | Go | Neuronové sítě + MCTS |
| 2019 | AlphaStar | StarCraft II | Reinforcement Learning |

### Tradiční vs. Moderní AI

**Tradiční AI (Minimax):**
- Prohledává strom všech možných tahů
- Předvídatelná, ale neporazitelná v jednoduchých hrách
- Problém: exponenciální růst možností

**Moderní AI (Reinforcement Learning):**
- Učí se hraním sama proti sobě
- Objevuje nové strategie
- Zvládá komplexní hry (Go, StarCraft)

In [ ]:
# 📊 Vizualizace: Složitost her

hry = ['Piškvorky\n(3x3)', 'Dáma', 'Šachy', 'Go\n(19x19)']
moznosti = [9, 500, 10**44, 10**170]  # Přibližný počet možných pozic
log_moznosti = [np.log10(m) if m > 0 else 0 for m in [9, 500, 1e44, 1e170]]

fig, ax = plt.subplots(figsize=(12, 6))

colors = ['#2ecc71', '#3498db', '#9b59b6', '#e74c3c']
bars = ax.bar(hry, log_moznosti, color=colors, edgecolor='black', linewidth=2)

ax.set_ylabel('log₁₀(počet možných pozic)', fontsize=12)
ax.set_title('🎮 Složitost deskových her - Proč je Go tak těžké?', 
             fontsize=14, fontweight='bold')

# Přidáme hodnoty
labels = ['9', '~500', '10⁴⁴', '10¹⁷⁰']
for bar, label in zip(bars, labels):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
            label, ha='center', va='bottom', fontsize=12, fontweight='bold')

# Přidáme horizontální čáry pro porovnání
ax.axhline(y=80, color='red', linestyle='--', alpha=0.5)
ax.text(3.5, 82, 'Atomy ve vesmíru (~10⁸⁰)', fontsize=10, color='red')

ax.set_ylim(0, 180)
plt.tight_layout()
plt.show()

print("\n💡 Go má více možných pozic než atomů ve vesmíru!")
print("   Proto tradiční Minimax nefunguje - potřebujeme neuronové sítě.")

---

## 📖 Část 2: Minimax algoritmus - Teorie

### Jak Minimax přemýšlí?

AI je **extrémní pesimista**:
1. Předpokládá, že soupeř vždy zahraje **nejlepší možný tah**
2. Vybírá tah, který **minimalizuje maximální** škodu
3. Prohledává **strom všech možných tahů**

### Hodnocení pozic:
- **+10** = AI vyhrála
- **-10** = Hráč vyhrál
- **0** = Remíza

### Vizualizace stromu rozhodování:

```
                    [AI volí MAX]
                    /     |     \
               [Hráč volí MIN]  [Hráč volí MIN]  [Hráč volí MIN]
               /    \        /    \        /    \
            [+10] [-10]   [0]   [+10]   [-10]  [0]
```

AI vybere cestu, kde i při nejlepším tahu soupeře nedojde k prohře.

In [ ]:
# 🌳 Vizualizace Minimax stromu

fig, ax = plt.subplots(figsize=(14, 10))

# Pozice uzlů
pozice = {
    'root': (0.5, 0.9),
    'L1_1': (0.2, 0.6),
    'L1_2': (0.5, 0.6),
    'L1_3': (0.8, 0.6),
    'L2_1': (0.1, 0.3),
    'L2_2': (0.3, 0.3),
    'L2_3': (0.4, 0.3),
    'L2_4': (0.6, 0.3),
    'L2_5': (0.7, 0.3),
    'L2_6': (0.9, 0.3)
}

# Hodnoty listů
hodnoty = {
    'L2_1': ('+10', '#2ecc71'),  # Výhra AI
    'L2_2': ('-10', '#e74c3c'),  # Prohra AI
    'L2_3': ('0', '#f39c12'),    # Remíza
    'L2_4': ('+10', '#2ecc71'),
    'L2_5': ('-10', '#e74c3c'),
    'L2_6': ('0', '#f39c12')
}

# Kreslíme uzly
def kresli_uzel(ax, pos, text, color, size=0.08):
    circle = plt.Circle(pos, size, color=color, ec='black', lw=2)
    ax.add_patch(circle)
    ax.text(pos[0], pos[1], text, ha='center', va='center', 
            fontsize=12, fontweight='bold', color='white')

# Root (AI volí MAX)
kresli_uzel(ax, pozice['root'], 'MAX\n(AI)', '#3498db')
ax.text(pozice['root'][0], pozice['root'][1] + 0.12, 
        '🤖 AI vybírá nejlepší tah', ha='center', fontsize=11)

# Level 1 (Hráč volí MIN)
for key in ['L1_1', 'L1_2', 'L1_3']:
    kresli_uzel(ax, pozice[key], 'MIN\n(Hráč)', '#9b59b6')

ax.text(0.02, 0.6, '👤 Hráč vybírá\nnejhorší pro AI', fontsize=10, va='center')

# Level 2 (Listy - koncové stavy)
for key, (val, col) in hodnoty.items():
    kresli_uzel(ax, pozice[key], val, col, size=0.06)

ax.text(0.02, 0.3, '🏁 Koncové\nstavy hry', fontsize=10, va='center')

# Kreslíme hrany
hrany = [
    ('root', 'L1_1'), ('root', 'L1_2'), ('root', 'L1_3'),
    ('L1_1', 'L2_1'), ('L1_1', 'L2_2'),
    ('L1_2', 'L2_3'), ('L1_2', 'L2_4'),
    ('L1_3', 'L2_5'), ('L1_3', 'L2_6')
]

for start, end in hrany:
    ax.plot([pozice[start][0], pozice[end][0]], 
            [pozice[start][1] - 0.08, pozice[end][1] + 0.08],
            'k-', lw=1.5, alpha=0.6)

# Výpočet hodnot pro vnitřní uzly
ax.annotate('min(-10, +10) = -10', xy=(0.2, 0.45), fontsize=9, 
            ha='center', color='#9b59b6')
ax.annotate('min(0, +10) = 0', xy=(0.5, 0.45), fontsize=9, 
            ha='center', color='#9b59b6')
ax.annotate('min(-10, 0) = -10', xy=(0.8, 0.45), fontsize=9, 
            ha='center', color='#9b59b6')

# Výsledek root
ax.annotate('max(-10, 0, -10) = 0', xy=(0.5, 0.75), fontsize=10, 
            ha='center', color='#3498db', fontweight='bold')
ax.annotate('→ AI volí prostřední tah!', xy=(0.72, 0.75), fontsize=10, 
            ha='left', color='green', fontweight='bold')

# Legenda
legend_elements = [
    mpatches.Patch(color='#2ecc71', label='Výhra AI (+10)'),
    mpatches.Patch(color='#e74c3c', label='Prohra AI (-10)'),
    mpatches.Patch(color='#f39c12', label='Remíza (0)')
]
ax.legend(handles=legend_elements, loc='lower right')

ax.set_xlim(-0.05, 1.05)
ax.set_ylim(0.15, 1.05)
ax.axis('off')
ax.set_title('🌳 Minimax rozhodovací strom', fontsize=16, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 AI vybírá tah, který zaručuje nejméně 0 (remízu),")
print("   i když soupeř hraje optimálně!")

---

## 📖 Část 3: Implementace piškvorek

Nyní naprogramujeme kompletní hru **Piškvorky (Tic-Tac-Toe)** s neporazitelnou AI!

In [ ]:
# 🎮 PIŠKVORKY - Základní struktura

# Konstanty pro hráče
HRAC_X = "X"  # Lidský hráč
HRAC_O = "O"  # AI
PRAZDNE = " "

def vytvor_desku():
    """Vytvoří prázdnou herní desku 3x3."""
    return [[PRAZDNE for _ in range(3)] for _ in range(3)]

def zobraz_desku(deska):
    """Zobrazí aktuální stav herní desky."""
    print("\n    0   1   2")
    print("  ┌───┬───┬───┐")
    for i, radek in enumerate(deska):
        print(f"{i} │ {radek[0]} │ {radek[1]} │ {radek[2]} │")
        if i < 2:
            print("  ├───┼───┼───┤")
    print("  └───┴───┴───┘")

def zobraz_desku_graficky(deska, ax=None):
    """Grafické zobrazení herní desky."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 6))
    
    ax.clear()
    
    # Mřížka
    for i in range(1, 3):
        ax.axhline(y=i, color='black', linewidth=2)
        ax.axvline(x=i, color='black', linewidth=2)
    
    # Symboly
    for i in range(3):
        for j in range(3):
            symbol = deska[i][j]
            if symbol == HRAC_X:
                ax.text(j + 0.5, 2.5 - i, 'X', fontsize=50, 
                       ha='center', va='center', color='#e74c3c', fontweight='bold')
            elif symbol == HRAC_O:
                ax.text(j + 0.5, 2.5 - i, 'O', fontsize=50, 
                       ha='center', va='center', color='#3498db', fontweight='bold')
    
    ax.set_xlim(0, 3)
    ax.set_ylim(0, 3)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('🎮 Piškvorky', fontsize=16)
    
    return ax

# Test
deska = vytvor_desku()
print("📋 Prázdná herní deska:")
zobraz_desku(deska)

# Přidáme nějaké tahy pro ukázku
deska[1][1] = HRAC_X
deska[0][0] = HRAC_O
deska[2][2] = HRAC_X

print("\n📋 Deska po několika tazích:")
zobraz_desku(deska)

# Grafické zobrazení
fig, ax = plt.subplots(figsize=(5, 5))
zobraz_desku_graficky(deska, ax)
plt.show()

In [ ]:
# 🏆 Funkce pro kontrolu výhry a remízy

def je_vyhra(deska, hrac):
    """Zkontroluje, zda daný hráč vyhrál."""
    # Kontrola řádků
    for radek in deska:
        if all(cell == hrac for cell in radek):
            return True
    
    # Kontrola sloupců
    for sloupec in range(3):
        if all(deska[radek][sloupec] == hrac for radek in range(3)):
            return True
    
    # Kontrola diagonál
    if all(deska[i][i] == hrac for i in range(3)):
        return True
    if all(deska[i][2-i] == hrac for i in range(3)):
        return True
    
    return False

def je_remiza(deska):
    """Zkontroluje, zda je hra remíza (všechna pole jsou plná)."""
    return all(cell != PRAZDNE for radek in deska for cell in radek)

def ziskej_mozne_tahy(deska):
    """Vrátí seznam všech možných (prázdných) tahů."""
    tahy = []
    for i in range(3):
        for j in range(3):
            if deska[i][j] == PRAZDNE:
                tahy.append((i, j))
    return tahy

# Test funkcí
print("🧪 TEST FUNKCÍ")
print("="*40)

# Test výhry v řádku
test_deska1 = [[HRAC_X, HRAC_X, HRAC_X],
               [PRAZDNE, HRAC_O, PRAZDNE],
               [HRAC_O, PRAZDNE, PRAZDNE]]
print(f"\nŘádková výhra X: {je_vyhra(test_deska1, HRAC_X)}")
zobraz_desku(test_deska1)

# Test výhry v diagonále
test_deska2 = [[HRAC_O, HRAC_X, PRAZDNE],
               [HRAC_X, HRAC_O, PRAZDNE],
               [PRAZDNE, PRAZDNE, HRAC_O]]
print(f"\nDiagonální výhra O: {je_vyhra(test_deska2, HRAC_O)}")
zobraz_desku(test_deska2)

# Test možných tahů
print(f"\nMožné tahy na druhé desce: {ziskej_mozne_tahy(test_deska2)}")

---

## 📖 Část 4: Minimax algoritmus - Implementace

Toto je **mozek** naší AI! Funkce rekurzivně zkouší všechny tahy a hodnotí je.

In [ ]:
# 🧠 MINIMAX ALGORITMUS

def minimax(deska, hloubka, je_tah_ai):
    """
    Minimax algoritmus pro nalezení nejlepšího tahu.
    
    Args:
        deska: Aktuální stav herní desky
        hloubka: Hloubka rekurze (pro penalizaci delších her)
        je_tah_ai: True pokud je na tahu AI, False pro hráče
    
    Returns:
        Skóre pozice (-10 až +10)
    """
    # Základní případy (terminální stavy)
    if je_vyhra(deska, HRAC_O):  # AI (O) vyhrála
        return 10 - hloubka  # Preferujeme rychlejší výhru
    if je_vyhra(deska, HRAC_X):  # Hráč (X) vyhrál
        return hloubka - 10  # Preferujeme pomalejší prohru
    if je_remiza(deska):
        return 0
    
    if je_tah_ai:  # AI (maximalizující hráč)
        nejlepsi_skore = float('-inf')
        for tah in ziskej_mozne_tahy(deska):
            # Simulujeme tah
            deska[tah[0]][tah[1]] = HRAC_O
            # Rekurzivně hodnotíme
            skore = minimax(deska, hloubka + 1, False)
            # Vrátíme tah zpět
            deska[tah[0]][tah[1]] = PRAZDNE
            nejlepsi_skore = max(nejlepsi_skore, skore)
        return nejlepsi_skore
    
    else:  # Hráč (minimalizující hráč)
        nejlepsi_skore = float('inf')
        for tah in ziskej_mozne_tahy(deska):
            # Simulujeme tah
            deska[tah[0]][tah[1]] = HRAC_X
            # Rekurzivně hodnotíme
            skore = minimax(deska, hloubka + 1, True)
            # Vrátíme tah zpět
            deska[tah[0]][tah[1]] = PRAZDNE
            nejlepsi_skore = min(nejlepsi_skore, skore)
        return nejlepsi_skore

def najdi_nejlepsi_tah(deska):
    """
    Najde nejlepší možný tah pro AI.
    
    Returns:
        Tuple (radek, sloupec) nejlepšího tahu
    """
    nejlepsi_skore = float('-inf')
    nejlepsi_tah = None
    
    print("\n🤖 AI analyzuje možné tahy:")
    
    for tah in ziskej_mozne_tahy(deska):
        # Simulujeme tah
        deska[tah[0]][tah[1]] = HRAC_O
        # Spočítáme skóre
        skore = minimax(deska, 0, False)
        # Vrátíme tah zpět
        deska[tah[0]][tah[1]] = PRAZDNE
        
        print(f"   Tah {tah}: skóre = {skore}")
        
        if skore > nejlepsi_skore:
            nejlepsi_skore = skore
            nejlepsi_tah = tah
    
    print(f"\n✅ AI vybírá tah: {nejlepsi_tah} (skóre: {nejlepsi_skore})")
    return nejlepsi_tah

# Test Minimax algoritmu
print("🧪 TEST MINIMAX ALGORITMU")
print("="*50)

test_deska = [[HRAC_X, HRAC_O, HRAC_X],
              [PRAZDNE, HRAC_X, PRAZDNE],
              [PRAZDNE, PRAZDNE, HRAC_O]]

print("\nAktuální stav desky:")
zobraz_desku(test_deska)

nejlepsi = najdi_nejlepsi_tah(test_deska)

---

## 📖 Část 5: Kompletní hra

Spojíme vše dohromady a vytvoříme interaktivní hru!

In [ ]:
# 🎮 KOMPLETNÍ HRA - HRÁČ vs AI

def hraj_piskvorky():
    """Hlavní funkce pro spuštění hry."""
    deska = vytvor_desku()
    
    print("\n" + "="*50)
    print("🎮 PIŠKVORKY - HRÁČ (X) vs. AI (O)")
    print("="*50)
    print("\nZadávejte souřadnice jako: řádek sloupec")
    print("Příklad: 1 1 = střed desky")
    print("\nVyhrajte, pokud dokážete! 😎")
    
    while True:
        zobraz_desku(deska)
        
        # Tah hráče
        while True:
            try:
                vstup = input("\n👤 Váš tah (řádek sloupec): ").strip()
                radek, sloupec = map(int, vstup.split())
                
                if not (0 <= radek <= 2 and 0 <= sloupec <= 2):
                    print("❌ Souřadnice musí být 0-2!")
                    continue
                    
                if deska[radek][sloupec] != PRAZDNE:
                    print("❌ Toto pole je již obsazené!")
                    continue
                    
                deska[radek][sloupec] = HRAC_X
                break
                
            except (ValueError, IndexError):
                print("❌ Neplatný vstup! Zadejte dvě čísla oddělená mezerou.")
        
        # Kontrola výhry hráče
        if je_vyhra(deska, HRAC_X):
            zobraz_desku(deska)
            print("\n🎉 GRATULACE! Porazili jste AI!")
            print("   (To se normálně nestává... 🤔)")
            break
        
        # Kontrola remízy
        if je_remiza(deska):
            zobraz_desku(deska)
            print("\n🤝 REMÍZA! Dobrá hra!")
            break
        
        # Tah AI
        print("\n🤖 AI přemýšlí...")
        time.sleep(0.5)  # Dramatická pauza
        
        tah_ai = najdi_nejlepsi_tah(deska)
        if tah_ai:
            deska[tah_ai[0]][tah_ai[1]] = HRAC_O
        
        # Kontrola výhry AI
        if je_vyhra(deska, HRAC_O):
            zobraz_desku(deska)
            print("\n🤖 AI VYHRÁLA!")
            print("   Minimax je prostě neporazitelný...")
            break
        
        # Kontrola remízy
        if je_remiza(deska):
            zobraz_desku(deska)
            print("\n🤝 REMÍZA! Dobrá hra!")
            break

# Spuštění hry (odkomentujte pro hraní)
# hraj_piskvorky()

In [ ]:
# 🎮 DEMONSTRACE: AI vs AI

def ai_vs_ai():
    """Demonstrace hry AI proti AI."""
    deska = vytvor_desku()
    
    print("\n" + "="*50)
    print("🤖 AI vs AI - Demonstrace Minimax algoritmu")
    print("="*50)
    
    tahy = 0
    na_tahu = HRAC_X  # X začíná
    
    while True:
        tahy += 1
        print(f"\n--- Tah {tahy} ({na_tahu}) ---")
        
        # AI najde nejlepší tah
        nejlepsi_skore = float('-inf') if na_tahu == HRAC_O else float('inf')
        nejlepsi_tah = None
        
        for tah in ziskej_mozne_tahy(deska):
            deska[tah[0]][tah[1]] = na_tahu
            
            if na_tahu == HRAC_O:
                skore = minimax(deska, 0, False)
                if skore > nejlepsi_skore:
                    nejlepsi_skore = skore
                    nejlepsi_tah = tah
            else:
                skore = minimax(deska, 0, True)
                if skore < nejlepsi_skore:
                    nejlepsi_skore = skore
                    nejlepsi_tah = tah
            
            deska[tah[0]][tah[1]] = PRAZDNE
        
        deska[nejlepsi_tah[0]][nejlepsi_tah[1]] = na_tahu
        print(f"{na_tahu} hraje: {nejlepsi_tah}")
        zobraz_desku(deska)
        
        # Kontrola konce hry
        if je_vyhra(deska, na_tahu):
            print(f"\n🏆 {na_tahu} vyhrál!")
            break
        if je_remiza(deska):
            print("\n🤝 REMÍZA!")
            break
        
        # Střídání hráčů
        na_tahu = HRAC_O if na_tahu == HRAC_X else HRAC_X
    
    return deska

# Spustíme demonstraci
vysledna_deska = ai_vs_ai()

print("\n💡 Když hrají dva dokonalí hráči (Minimax), vždy je REMÍZA!")

---

## 📖 Část 6: Moderní herní AI - Reinforcement Learning

### Od pravidel k učení

Moderní AI (jako AlphaGo) nepoužívá Minimax. Místo toho se **učí hraním**:

```
┌─────────────────────────────────────────────────────────┐
│                  REINFORCEMENT LEARNING                 │
├─────────────────────────────────────────────────────────┤
│                                                         │
│   Agent (AI) ──→ Akce ──→ Prostředí (hra)              │
│       ↑                        │                        │
│       │                        │                        │
│       └──── Odměna/Trest ←─────┘                        │
│                                                         │
│   Cíl: Maximalizovat celkovou odměnu                   │
└─────────────────────────────────────────────────────────┘
```

### Jak se AI učí hrát?

1. **Náhodné hraní** - Na začátku AI hraje náhodně
2. **Odměny** - Za výhru +1, za prohru -1
3. **Učení** - AI upravuje strategii podle odměn
4. **Self-play** - Hraje sama proti sobě milionkrát

In [ ]:
# 📊 Vizualizace: Minimax vs Reinforcement Learning

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Graf 1: Minimax
ax1 = axes[0]
kroky_minimax = ['Definuj\npravidla', 'Prohledej\nstrom', 'Vyber\nnejlepší tah']
y_pos = [0.7, 0.4, 0.1]

for i, (krok, y) in enumerate(zip(kroky_minimax, y_pos)):
    rect = mpatches.FancyBboxPatch((0.1, y), 0.8, 0.2,
                                    boxstyle="round,pad=0.02",
                                    facecolor='#3498db', edgecolor='black')
    ax1.add_patch(rect)
    ax1.text(0.5, y + 0.1, krok, ha='center', va='center', 
             fontsize=11, color='white', fontweight='bold')
    if i < 2:
        ax1.annotate('', xy=(0.5, y), xytext=(0.5, y + 0.1),
                    arrowprops=dict(arrowstyle='->', color='black', lw=2))

ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)
ax1.axis('off')
ax1.set_title('🎯 Minimax\n(Rule-based)', fontsize=14, fontweight='bold')

# Graf 2: Reinforcement Learning
ax2 = axes[1]

# Cyklus učení
circle = plt.Circle((0.5, 0.5), 0.35, fill=False, color='#2ecc71', linewidth=3)
ax2.add_patch(circle)

# Pozice na kruhu
pozice_rl = [
    (0.5, 0.85, 'Stav'),
    (0.85, 0.5, 'Akce'),
    (0.5, 0.15, 'Odměna'),
    (0.15, 0.5, 'Učení')
]

for x, y, text in pozice_rl:
    circle_small = plt.Circle((x, y), 0.1, color='#2ecc71', ec='black', lw=2)
    ax2.add_patch(circle_small)
    ax2.text(x, y, text, ha='center', va='center', 
             fontsize=10, color='white', fontweight='bold')

# Šipky
arrow_style = dict(arrowstyle='->', color='#2ecc71', lw=2)
ax2.annotate('', xy=(0.78, 0.65), xytext=(0.58, 0.78), arrowprops=arrow_style)
ax2.annotate('', xy=(0.78, 0.35), xytext=(0.78, 0.58), arrowprops=arrow_style)
ax2.annotate('', xy=(0.42, 0.22), xytext=(0.7, 0.25), arrowprops=arrow_style)
ax2.annotate('', xy=(0.22, 0.42), xytext=(0.22, 0.22), arrowprops=arrow_style)

ax2.text(0.5, 0.5, '🔄', fontsize=30, ha='center', va='center')

ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.axis('off')
ax2.set_title('🧠 Reinforcement Learning\n(Self-learning)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 SROVNÁNÍ:")
print("┌──────────────────────┬──────────────────────┬──────────────────────┐")
print("│ Vlastnost            │ Minimax              │ Reinforcement L.     │")
print("├──────────────────────┼──────────────────────┼──────────────────────┤")
print("│ Potřebuje pravidla   │ ✅ Ano               │ ❌ Ne                │")
print("│ Učí se               │ ❌ Ne                │ ✅ Ano               │")
print("│ Škálovatelnost       │ ❌ Omezená           │ ✅ Vysoká            │")
print("│ Překvapivé strategie │ ❌ Ne                │ ✅ Ano               │")
print("│ Vhodné pro           │ Jednoduché hry       │ Komplexní hry        │")
print("└──────────────────────┴──────────────────────┴──────────────────────┘")

---

## 🎯 Mini-projekt: Vylepšete AI!

### Úkoly pro vás:

1. **Alpha-Beta prořezávání** - Optimalizujte Minimax algoritmus
2. **Větší deska** - Rozšiřte hru na 4x4 nebo 5x5
3. **Různé úrovně obtížnosti** - Přidejte náhodnost pro lehčí AI
4. **GUI** - Vytvořte grafické rozhraní s tlačítky

In [ ]:
# 🎯 BONUS: Alpha-Beta prořezávání

def minimax_alphabeta(deska, hloubka, alpha, beta, je_tah_ai):
    """
    Minimax s Alpha-Beta prořezáváním - rychlejší verze!
    
    Alpha: Nejlepší hodnota, kterou může MAX (AI) garantovat
    Beta: Nejlepší hodnota, kterou může MIN (hráč) garantovat
    """
    # Základní případy
    if je_vyhra(deska, HRAC_O):
        return 10 - hloubka
    if je_vyhra(deska, HRAC_X):
        return hloubka - 10
    if je_remiza(deska):
        return 0
    
    if je_tah_ai:  # MAX
        nejlepsi_skore = float('-inf')
        for tah in ziskej_mozne_tahy(deska):
            deska[tah[0]][tah[1]] = HRAC_O
            skore = minimax_alphabeta(deska, hloubka + 1, alpha, beta, False)
            deska[tah[0]][tah[1]] = PRAZDNE
            
            nejlepsi_skore = max(nejlepsi_skore, skore)
            alpha = max(alpha, skore)
            
            # Prořezání!
            if beta <= alpha:
                break
                
        return nejlepsi_skore
    
    else:  # MIN
        nejlepsi_skore = float('inf')
        for tah in ziskej_mozne_tahy(deska):
            deska[tah[0]][tah[1]] = HRAC_X
            skore = minimax_alphabeta(deska, hloubka + 1, alpha, beta, True)
            deska[tah[0]][tah[1]] = PRAZDNE
            
            nejlepsi_skore = min(nejlepsi_skore, skore)
            beta = min(beta, skore)
            
            # Prořezání!
            if beta <= alpha:
                break
                
        return nejlepsi_skore

# Porovnání rychlosti
import time

test_deska = vytvor_desku()
test_deska[1][1] = HRAC_X  # Jeden tah

# Měření běžného Minimax
start = time.time()
for _ in range(10):
    for tah in ziskej_mozne_tahy(test_deska):
        test_deska[tah[0]][tah[1]] = HRAC_O
        minimax(test_deska, 0, False)
        test_deska[tah[0]][tah[1]] = PRAZDNE
cas_minimax = time.time() - start

# Měření Alpha-Beta
start = time.time()
for _ in range(10):
    for tah in ziskej_mozne_tahy(test_deska):
        test_deska[tah[0]][tah[1]] = HRAC_O
        minimax_alphabeta(test_deska, 0, float('-inf'), float('inf'), False)
        test_deska[tah[0]][tah[1]] = PRAZDNE
cas_alphabeta = time.time() - start

print("⏱️ SROVNÁNÍ RYCHLOSTI:")
print(f"   Minimax:        {cas_minimax:.4f}s")
print(f"   Alpha-Beta:     {cas_alphabeta:.4f}s")
print(f"   Zrychlení:      {cas_minimax/cas_alphabeta:.1f}x")

In [ ]:
# 🧪 Prostor pro vaše experimenty

# Zde můžete experimentovat s vlastními vylepšeními!
# Zkuste:
# - Přidat různé úrovně obtížnosti
# - Vytvořit větší herní desku
# - Implementovat jiné hry (Connect 4, Reversi)



---

## 📝 Shrnutí kapitoly

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white; margin: 30px 0;">
    <h3 style="color: white; margin-top: 0;">✅ Co jste se naučili</h3>
    <ul style="list-style: none; padding-left: 0;">
        <li>✓ Historie herní AI - od Pac-Mana po AlphaGo</li>
        <li>✓ Minimax algoritmus - jak AI "přemýšlí" o tazích</li>
        <li>✓ Implementace neporazitelné AI pro piškvorky</li>
        <li>✓ Alpha-Beta prořezávání - optimalizace Minimaxu</li>
        <li>✓ Základy Reinforcement Learning</li>
    </ul>
    
    <h3 style="color: white;">🔑 Klíčové pojmy</h3>
    <ul>
        <li><b>Minimax</b> - algoritmus pro hry dvou hráčů s nulovým součtem</li>
        <li><b>Alpha-Beta</b> - optimalizace, která ořezává zbytečné větve</li>
        <li><b>Reinforcement Learning</b> - učení pomocí odměn a trestů</li>
        <li><b>Self-play</b> - AI hraje sama proti sobě</li>
    </ul>
    
    <h3 style="color: white;">➡️ Další kapitola</h3>
    <p>V kapitole 9 vytvoříme <b>mini-projekt s Teachable Machine</b>!</p>
</div>

---

*📅 Notebook vytvořen: 2025*  
*🎓 Kurz: AI pro každého - Od základů k praxi*  
*🔧 Prostředí: Google Colab (Python 3)*